In [11]:
import scrapbook as sb
import pandas as pd
from IPython.display import HTML

# List of notebook paths
notebook_paths = [
    "notebooks/storm-2017_08_28.ipynb",
    "notebooks/storm-1986_03_11.ipynb",
    "notebooks/storm-2015_10_24.ipynb",
    "notebooks/storm-1989_05_14.ipynb",
]


def determine_storm_type(row):
    max_value = row.max()
    max_columns = row[row == max_value].index.tolist()
    if len(max_columns) > 1:
        return "hybrid"
    else:
        return max_columns[0]


def process_notebooks(notebook_paths):
    all_data = pd.DataFrame()
    for path in notebook_paths:
        storm_date = path.split("/")[-1].split(".")[0].split("-")[-1]
        nb = sb.read_notebook(path)
        scraps = nb.scraps
        stats_df = pd.DataFrame(scraps["summary_stats"].data)
        storm_type_sums = stats_df.sum(axis=1)
        df = storm_type_sums.to_frame().T
        df.insert(0, "storm_start_date", storm_date)
        df["likely_storm_type"] = df.drop("storm_start_date", axis=1).apply(
            determine_storm_type, axis=1
        )
        all_data = pd.concat([all_data, df], ignore_index=True)
    return all_data


def convert_paths_to_html_links(data_frame, notebook_paths):
    html_paths = [path.replace(".ipynb", ".html") for path in notebook_paths]
    data_frame["HTML_Link"] = html_paths
    data_frame["HTML_Link"] = data_frame["HTML_Link"].apply(
        lambda x: f"<a href='{x}'>{x}</a>"
    )
    return data_frame


all_data = process_notebooks(notebook_paths)
all_data_with_links = convert_paths_to_html_links(all_data, notebook_paths)
HTML(all_data_with_links.to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,notes,tropical,likely_storm_type,HTML_Link
0,2017_08_28,0.0,0.0,1.0,0.0,6.0,tropical,notebooks/storm-2017_08_28.html
1,1986_03_11,0.0,2.0,6.0,0.0,0.0,mid_latitude_cyclone,notebooks/storm-1986_03_11.html
2,2015_10_24,0.0,6.0,6.0,0.0,0.0,hybrid,notebooks/storm-2015_10_24.html
3,1989_05_14,0.0,6.0,1.0,0.0,0.0,mesoscale_storm,notebooks/storm-1989_05_14.html


## Mid-Latitude Cyclone

In [8]:
HTML(
    all_data[all_data["likely_storm_type"] == "mid_latitude_cyclone"].to_html(
        escape=False
    )
)

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,notes,tropical,likely_storm_type,HTML_Link
1,1986_03_11,0.0,2.0,6.0,0.0,0.0,mid_latitude_cyclone,notebooks/storm-1986_03_11.html


## Mesoscale Convective

In [9]:
HTML(all_data[all_data["likely_storm_type"] == "mesoscale_storm"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,notes,tropical,likely_storm_type,HTML_Link
3,1989_05_14,0.0,6.0,1.0,0.0,0.0,mesoscale_storm,notebooks/storm-1989_05_14.html


## Tropical

In [4]:
HTML(all_data[all_data["likely_storm_type"] == "tropical"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,notes,tropical,likely_storm_type,HTML_Link
0,2017_08_28,0.0,0.0,1.0,0.0,6.0,tropical,notebooks/storm-2017_08_28.html


## Local Storms

In [5]:
HTML(all_data[all_data["likely_storm_type"] == "local_storm"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,notes,tropical,likely_storm_type,HTML_Link


## Hybrid

In [6]:
HTML(all_data[all_data["likely_storm_type"] == "hybrid"].to_html(escape=False))

,storm_start_date,local_storm,mesoscale_storm,mid_latitude_cyclone,notes,tropical,likely_storm_type,HTML_Link
2,2015_10_24,0.0,6.0,6.0,0.0,0.0,hybrid,notebooks/storm-2015_10_24.html
